In [ ]:
from datetime import datetime

import metpy.calc as mpcalc
from metpy.plots import declarative
from metpy.units import units
import numpy as np
from siphon.catalog import TDSCatalog
import xarray as xr

In [ ]:
def get_satellite_file_url(datasets, date):
    '''A function to help find the desired satellite data based on the time given.
    
    Input:
     - List of datasets from a THREDDS Catalog
     - Date of desired dataset (datetime object)
     
    Output:
     - Index value of dataset closest to desired time
    '''
    sat_date_hour = date.strftime('%Y%j%H')
    files = []
    times = []
    for file in cat.datasets:
        if sat_date_hour in file:
            times.append(datetime.strptime(file.split('_')[3][1:-3], '%Y%j%H%M'))
            files.append(file)
    find_file = np.abs(np.array(times) - date)
    return list(cat.datasets).index(files[np.argmin(find_file)])

In [ ]:
date = datetime(2022, 3, 25, 12, 0)

# Create variables for URL generation
region = 'CONUS'
channel = 9
satellite = 'east'

# Construct the data_url string
data_url = (f'https://thredds.ucar.edu/thredds/catalog/satellite/goes/{satellite}/products/'
            f'CloudAndMoistureImagery/{region}/Channel{channel:02d}/'
            f'{date:%Y%m%d}/catalog.xml')

# Get list of files available for particular day
cat = TDSCatalog(data_url)

# Use homemade function to get dataset for desired time
dataset = cat.datasets[get_satellite_file_url(cat.datasets, date)]

# Open most recent file available
ds = dataset.remote_access(use_xarray=True)

# Apply a square root correction for visible imagery only
if channel == 2:
    ds['Sectorized_CMI'].values = np.sqrt(ds['Sectorized_CMI'].values)

# Grab time from file and make into datetime object for plotting and later data access
vtime = ds.time.values.astype('datetime64[ms]').astype('O')

In [ ]:
# Remote access to the dataset from the UCAR site
ds_gfs = xr.open_dataset('https://thredds.ucar.edu/thredds/dodsC/grib'
                         f'/NCEP/GFS/Global_onedeg/GFS_Global_onedeg_{date:%Y%m%d}_{date:%H%M}.grib2')

# Subset data to be just over the U.S. for plotting purposes
ds_gfs = ds_gfs.sel(lat=slice(70,10), lon=slice(360-150, 360-55)).metpy.parse_cf()